### 安裝mysql套件

In [ ]:
#pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pymysql
pymysql.__version__

'1.4.6'

### 連接mysql

In [24]:
connect=pymysql.connect(
user="root",
password="LIsy758eKell$",
host="127.0.0.1", #localhost
database="shop",
port=3306
)

connect

In [25]:
cursor=connect.cursor()
cursor

### 查詢語法

- select from tablename

In [56]:
sqlstr="select id, email,phone,address from user;"
cursor.execute(sqlstr)
#users=cursor.fetchmany(3)
users=cursor.fetchone()
users

(1, 'conquerkwu@hotmail.com', '0933256199', '桃園市中壢區七合一街95之5號10樓')

### 插入語法

In [42]:
username=input("姓名:")
email=input("Email:")
password=input("密碼:")
phone=input("手機號碼:")
address=input("地址:")

sqlstr=f"insert into user (username,email,password,phone,address)\
      values('{username}','{email}','{password}','{phone}','{address}');"

sqlstr

"insert into user (username,email,password,phone,address)      values('','','','','');"

In [8]:
try:
    cursor.execute(sqlstr)
    connect.commit()
except Exception as e:
    print("寫入失敗",e)    

寫入失敗 name 'sqlstr' is not defined


進入登入

In [52]:
email=input("Email:")
password=input("密碼:")

sqlstr=f"select * from user where email='{email}' and password='{password}'"
sqlstr


"select * from user where email='' and password=''"

In [10]:
cursor.execute(sqlstr)
user=cursor.fetchone()
user

(1,
 'macbook air 2023 pro',
 35000.0,
 3,
 '256ssd 16g\r\n\r\n',
 datetime.date(2025, 3, 20))

In [11]:
if user is not None:
    print("登入成功!")
else:
    print("登入失敗!")

登入成功!


建立商品資料表(item)
- id     - int(AI)
- name   - varchar(100) - unique
- price  - float        
- qty    - int          - default:1
- info   - text         - 可以為空

In [21]:
# 只需要重新登入上面程式 但不要用客戶資料 因為這個要產品資料
def update_item():

    sqlstr="select * from item;"
    cursor.execute(sqlstr)
    items=cursor.fetchall()
    return items
   
update_item()

((1,
  'macbook air 2023 pro',
  35000.0,
  2,
  '256ssd 16g\r\n\r\n',
  datetime.date(2025, 3, 20)),
 (2,
  'asus zenbook 14" notebook',
  24900.0,
  0,
  '華碩筆電',
  datetime.date(2025, 3, 20)),
 (3, 'asus po-行動電源', 699.0, 8, '25W', datetime.date(2025, 3, 20)),
 (4, 'Samsung A55手機', 12900.0, 3, '紅色限量', datetime.date(2025, 3, 20)),
 (5, 'iphone 16pro手機', 36000.0, 3, '白色限量', datetime.date(2025, 3, 20)))

In [23]:
#connect 關閉客戶 才可以跑產品資料
connect.close()

In [28]:
cart=[]

while True:
    # 同步更新最新產品
    items=update_item()
    # 選單
    print("-"*100)
    for i,item in enumerate(items):
        print(f"{i+1}.商品名稱:{item[1]} 價格:{item[2]} 數量:{item[3]} 備註:{item[4]}")
    print("-"*100)

    choice=int(input("輸入購買編號(0:exit):")) 
    if choice==0:
        break

    # 選項<1 or >len(items) 輸入不正確
    if choice<1 or choice>len(items):
        print("輸入不正確")
        continue

    # 紀錄購買商品
    buy_item=items[choice-1]

    # 數量小於1(無庫存)
    if buy_item[3]<1:
        print(f"****{buy_item[1]} 目前無庫存")
        continue

    #加入購物車
    cart.append(buy_item)

    # 更新商品數量

    sqlstr=f"update item set qty={buy_item[3]-1} where id={buy_item[0]};"
    print(sqlstr)
    cursor.execute(sqlstr)
    connect.commit()

    print(f"===>{buy_item[1]} 加入購物車")

print("完成購買")      
print(cart)  

----------------------------------------------------------------------------------------------------
1.商品名稱:macbook air 2023 pro 價格:35000.0 數量:2 備註:256ssd 16g


2.商品名稱:asus zenbook 14" notebook 價格:24900.0 數量:0 備註:華碩筆電
3.商品名稱:asus po-行動電源 價格:699.0 數量:8 備註:25W
4.商品名稱:Samsung A55手機 價格:12900.0 數量:3 備註:紅色限量
5.商品名稱:iphone 16pro手機 價格:36000.0 數量:3 備註:白色限量
----------------------------------------------------------------------------------------------------
update item set qty=1 where id=1;
===>macbook air 2023 pro 加入購物車
----------------------------------------------------------------------------------------------------
1.商品名稱:macbook air 2023 pro 價格:35000.0 數量:1 備註:256ssd 16g


2.商品名稱:asus zenbook 14" notebook 價格:24900.0 數量:0 備註:華碩筆電
3.商品名稱:asus po-行動電源 價格:699.0 數量:8 備註:25W
4.商品名稱:Samsung A55手機 價格:12900.0 數量:3 備註:紅色限量
5.商品名稱:iphone 16pro手機 價格:36000.0 數量:3 備註:白色限量
----------------------------------------------------------------------------------------------------
update item set qty=7 where id=3;

送出訂單
- id
- user id
- item_id
- created_time

In [32]:
for item in cart:
    #print(item)
    sqlstr=f"insert into `order` (user_id,item_id) values({user[0]},{item[0]});"
    #print(sqlstr)
    cursor.execute(sqlstr)
    connect.commit()
    print("送出訂單!")


送出訂單!
送出訂單!
送出訂單!
送出訂單!


開始出貨

In [33]:
#找出user

sqlstr="select * from `order`; "
cursor.execute(sqlstr)
orders=cursor.fetchall()
orders

((5, 1, 1, datetime.datetime(2025, 3, 21, 15, 16, 53)),
 (6, 1, 3, datetime.datetime(2025, 3, 21, 15, 16, 53)),
 (7, 1, 3, datetime.datetime(2025, 3, 21, 15, 16, 53)),
 (8, 1, 4, datetime.datetime(2025, 3, 21, 15, 16, 53)),
 (9, 1, 1, datetime.datetime(2025, 3, 21, 15, 21, 47)),
 (10, 1, 3, datetime.datetime(2025, 3, 21, 15, 21, 47)),
 (11, 1, 3, datetime.datetime(2025, 3, 21, 15, 21, 47)),
 (12, 1, 4, datetime.datetime(2025, 3, 21, 15, 21, 47)))

In [34]:
user_id=orders[0][1]
user_id

1

In [35]:
sqlstr=f"select * from user where id={user_id}"
cursor.execute(sqlstr)
user=cursor.fetchone()
user

(1,
 None,
 '12345678',
 'conquerkwu@hotmail.com',
 '0933256199',
 '桃園市中壢區七合一街95之5號10樓',
 datetime.datetime(2025, 3, 19, 12, 59, 31))

In [36]:
sqlstr=f"select item_id from `order` where user_id={user_id}"
cursor.execute(sqlstr)
buy_items=cursor.fetchall()
buy_items


((1,), (3,), (3,), (4,), (1,), (3,), (3,), (4,))